In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from ast import literal_eval
import matplotlib.pyplot as plt

In [2]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

transactions_train['t_dat_datetime']  = pd.to_datetime(transactions_train['t_dat'])

transactions_train['t_month'] = transactions_train['t_dat_datetime'].dt.month

transactions_train['t_year']  = transactions_train['t_dat_datetime'].dt.year

transactions_train['customer_max_tran']  = transactions_train.\
groupby(['customer_id'])['t_dat_datetime'].transform('max')

In [11]:
cust_data = transactions_train[(transactions_train['customer_max_tran'].dt.year==2020)&
                                     (transactions_train['customer_max_tran'].dt.month == 9)]

# cust_data['article_id'] = cust_data.apply(lambda x :  'a'+ str(x.article_id), axis = 1)

def str_article(x):
    return 'a'+str(x)

cust_data['article_id'] = np.vectorize(str_article)(cust_data['article_id'])

In [14]:
cust_data['article_max_purchase_date'] = cust_data.groupby(['article_id'])['t_dat_datetime'].transform('max')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
cust_data['product_purchase'] = cust_data.groupby(['article_id'])['customer_id'].transform('nunique')

cust_data = cust_data[cust_data['product_purchase'] >= 12]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
cust_data['customer_visits'] = cust_data.groupby(['customer_id'])['t_dat'].transform('nunique')

cust_data = cust_data[cust_data['customer_visits'] >= 12]

In [18]:
cust_data['customer_prod_count'] = cust_data.groupby(['customer_id'])['article_id'].transform('nunique')

cust_data = cust_data[cust_data['customer_prod_count'] >= 4]

In [19]:
cust_data['product_max_purc_date'] = cust_data.groupby(['article_id'])['t_dat_datetime'].transform('max')

In [31]:
cust_data_2020_9 = cust_data[(cust_data['product_max_purc_date'].dt.year==2020)&
                                     (cust_data['product_max_purc_date'].dt.month == 9)]

In [32]:
cust_data_2020_9['product_purchase'] = cust_data_2020_9.groupby(['article_id'])['customer_id'].transform('nunique')

cust_data_2020_9 = cust_data_2020_9[cust_data_2020_9['product_purchase'] >= 12]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
cust_data_2020_9 = cust_data_2020_9.groupby(['customer_id','article_id']).agg(tran_count = ('t_dat_datetime','nunique')).\
assign(Rank = lambda x : x.groupby('customer_id')['tran_count'].rank(method='first', ascending=False)
      ).\
reset_index().\
query('Rank <= 12').\
groupby('customer_id').agg(prods = ('article_id','unique')).reset_index().\
assign(prods = lambda x:  list(x['prods']))


# cust_data_2020_9["prods"] = cust_data_2020_9["prods"].apply( lambda x : list(x) ) 

In [34]:
cust_data_2020_9['prods_len'] = cust_data_2020_9.apply(lambda x: len(x.prods) , axis =1)


In [35]:
article_df =  pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
customer_df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')

article_df['article_id'] = 'a'+article_df['article_id'].astype('str')

In [36]:
cust_data = pd.merge(cust_data,
                     cust_data_2020_9['customer_id'],
                     left_on = 'customer_id',
                     right_on = 'customer_id',
                    )

In [37]:
cust_data = pd.merge(pd.merge(cust_data ,
         article_df[['article_id','product_group_name']],
         left_on = 'article_id',
         right_on = 'article_id'
        ),
         customer_df[['customer_id','age']],
         left_on = 'customer_id',
         right_on = 'customer_id'
        )

In [38]:
cust_features = cust_data.groupby(['customer_id','age']).agg( tran_count = ('t_dat','nunique'),
                                       article_count = ('article_id','nunique'),
                                       spend = ('price','sum')
                                      ).reset_index()

In [39]:
cust_features_1 = pd.pivot_table(cust_data[['customer_id','product_group_name','t_dat']],
         index = 'customer_id',
         columns = 'product_group_name',
         values = 't_dat',
         aggfunc = 'nunique'
        ).fillna(0).reset_index()

In [40]:
cust_features = pd.merge(cust_features, 
                         cust_features_1,
                         left_on = 'customer_id',
                         right_on = 'customer_id',
                        )

In [41]:
# cust_data_2020_9["prods"] = cust_data_2020_9["prods"].apply( lambda x : list(x) ) 
# cust_data_2020_9["prods"].values[:5]

In [45]:
article_df['index_group_name'].unique()

array(['Ladieswear', 'Baby/Children', 'Menswear', 'Sport', 'Divided'],
      dtype=object)

In [54]:
cust_features.groupby(['age']).agg({'customer_id':'nunique'}).reset_index().\
sort_values('customer_id', ascending = False).head(60)

,age,customer_id
9,25.0,4994
8,24.0,4948
10,26.0,4718
7,23.0,4598
11,27.0,4294
6,22.0,3915
5,21.0,3705
12,28.0,3646
13,29.0,3279
14,30.0,3078


In [42]:
def age_groups(x):
    if x <= 10:
        'kids'
    elif x > 10 & x <= 20:
        'early_teen'
    elif x > 20  & x <= 25:
        'young'
    elif x > 25 &  x<= 30:
        'late_20'
    elif x > 30 & x<= 35:
        'early_30'
    elif x > 35 & x <= 40:
        'late_30'
    elif x > 40 & x <= 45:
        'early_40'
    elif x > 45 & x <= 50:
        'late_40'
    elif x > 50 & x <= 55:
        'early_50'
    elif x > 55 & x <= 60:
        'late_50'
    elif x > 60 & x <= 70:
        'early_60'
    elif x > 70 :
        'above_70'
    
        
cust_features.head()

,customer_id,age,tran_count,article_count,spend,Accessories,Bags,Cosmetic,Furniture,Garment Full body,...,Garment Upper body,Garment and Shoe care,Items,Nightwear,Shoes,Socks & Tights,Stationery,Swimwear,Underwear,Unknown
0,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,44.0,143,534,24.480153,23.0,0.0,0.0,0.0,55.0,...,87.0,0.0,0.0,5.0,14.0,7.0,0.0,10.0,14.0,1.0
1,0001f8cef6b9702d54abf66fd89eb21014bf98567065a9...,21.0,14,21,0.760288,3.0,0.0,0.0,0.0,1.0,...,6.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0
2,0002cca4cc68601e894ab62839428e5f0696417fe0f9e8...,55.0,20,45,2.082627,2.0,0.0,0.0,0.0,4.0,...,12.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
3,0006bb0fad5c49341bd9cece264271e68e01a4e55f22ec...,49.0,23,77,2.485831,1.0,0.0,0.0,0.0,2.0,...,18.0,0.0,0.0,1.0,2.0,0.0,0.0,3.0,1.0,0.0
4,0006bfcec82c6c132276c0a5549ae13645c9ab77b0243f...,19.0,15,30,0.934220,1.0,0.0,0.0,0.0,8.0,...,7.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [ ]:
model_data = pd.merge(cust_data_2020_9,
                      cust_features,
                      left_on = 'customer_id',
                      right_on = 'customer_id'
                     )

In [ ]:
import gc

del cust_data_2020_9
gc.collect()

del cust_features
gc.collect()

In [ ]:
del customer_df
gc.collect()


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [ ]:
genre_mlb = mlb.fit_transform(model_data['prods'])
genre_mlb

In [ ]:
len(genre_mlb[0:1][0])

In [ ]:
genre = mlb.inverse_transform(genre_mlb[0:1])
genre

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD

In [ ]:
model_data.shape[0]*0.8

In [ ]:
X_train = model_data.iloc[0:75000,2:]
y_train = genre_mlb[0:75000]
X_test = model_data.iloc[75000:,2:]
y_test = genre_mlb[75000:]

In [ ]:
model = Sequential()
model.add(Dense(5000, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(600, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

In [ ]:
# def macro_soft_f1(y, y_hat):
#     """Compute the macro soft F1-score as a cost.
#     Average (1 - soft-F1) across all labels.
#     Use probability values instead of binary predictions.
    
#     Args:
#         y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
#         y_hat (float32 Tensor): probability matrix of shape (BATCH_SIZE, N_LABELS)
        
#     Returns:
#         cost (scalar Tensor): value of the cost function for the batch
#     """
    
#     y = tf.cast(y, tf.float32)
#     y_hat = tf.cast(y_hat, tf.float32)
#     tp = tf.reduce_sum(y_hat * y, axis=0)
#     fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
#     fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
#     soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
#     cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
#     macro_cost = tf.reduce_mean(cost) # average on all labels
    
#     return macro_cost

In [ ]:
# def macro_f1(y, y_hat, thresh=0.7):
#     """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
#     Args:
#         y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
#         y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
#         thresh: probability value above which we predict positive
        
#     Returns:
#         macro_f1 (scalar Tensor): value of macro F1 for the batch
#     """
#     y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
#     tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
#     fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
#     fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
#     f1 = 2*tp / (2*tp + fn + fp + 1e-16)
#     macro_f1 = tf.reduce_mean(f1)
#     return macro_f1

In [ ]:
# correct_prediction = tf.equal(tf.round(tf.nn.sigmoid(pred)), tf.round(y_test))

# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# all_labels_true = tf.reduce_min(tf.cast(correct_prediction, tf.float32),1)
# accuracy2 = tf.reduce_mean(all_labels_true)

In [ ]:
# def accuracy2(y, y_hat):
    
#     correct_prediction = tf.equal(tf.round(tf.nn.sigmoid(y_hat)), tf.round(y))
   
#     all_labels_true = tf.reduce_min(tf.cast(correct_prediction, tf.float32),1)
    
#     accuracy2 = tf.reduce_mean(all_labels_true)
    
#     return accuracy2

In [ ]:
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss=macro_soft_f1,
#   metrics=[macro_f1], optimizer=sgd)

model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=['categorical_accuracy']
)

model.fit(X_train, y_train, epochs=10, batch_size=100)

In [ ]:
preds = model.predict(X_test)
preds[preds>=0.7] = 1
preds[preds<0.7] = 0
# score = compare preds and y_test

In [ ]:
len(preds[0])

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=10)
score

In [ ]:
genre_mlb[0:1].shape

In [ ]:
preds[0:1].astype(int).shape

In [ ]:
mlb.inverse_transform(genre_mlb[0:1])

In [ ]:
mlb.inverse_transform(preds[0:1].astype(int))

In [ ]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

submission['prediction'] = top_12_products.article_id.sum()

submission.to_csv('submission.csv', index = False)